<a href="https://colab.research.google.com/github/hinswhale/study/blob/master/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 11.2 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 3.3 MB 53.8 MB/s 
     |████████████████████████████████| 636 kB 28.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
! git clone https://github.com/hinswhale/study

Cloning into 'study'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [ ]:
!cd ..| ls -lh

total 8.0K
drwxr-xr-x 1 root root 4.0K Aug 13 13:35 sample_data
drwxr-xr-x 3 root root 4.0K Aug 17 09:55 study


In [ ]:
import torch
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
#%%
#part 1 - bert feature base
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as tfs
import warnings

warnings.filterwarnings('ignore')

In [ ]:
#%%
train_df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv',
                       delimiter='\t', header=None)

train_set = train_df[:3000]  #取其中的3000条数据作为我们的数据集
print("Train set shape:", train_set.shape)
train_set[1].value_counts()  #查看数据集中标签的分布
train_df.sample(10)


Train set shape: (3000, 2)


,0,1
317,a weak and ineffective ghost story without a c...,0
4308,it leaves little doubt that kidman has become ...,1
850,goldbacher draws on an elegant visual sense an...,1
2657,"plenty of warmth to go around , with music and...",1
3569,clint eastwood 's blood work is a lot like a w...,1
1719,at times a bit melodramatic and even a little ...,1
759,strictly a ` guy 's film ' in the worst sense ...,0
5261,my own minority report is that it stinks,0
3466,i can analyze this movie in three words thumbs...,0
2301,a generic family comedy unlikely to be appreci...,0


In [ ]:
model_class, tokenizer_class, pretrained_weights = (tfs.BertModel, tfs.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

#add_special_tokens 表示在句子的首尾添加[CLS]和[END]符号
train_tokenized = train_set[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
train_max_len = 0
for i in train_tokenized.values:
    if len(i) > train_max_len:
        train_max_len = len(i)

train_padded = np.array([i + [0] * (train_max_len-len(i)) for i in train_tokenized.values])
print("train set shape:",train_padded.shape)

#output：train set shape: (3000, 66)

train set shape: (3000, 66)


In [ ]:
train_attention_mask = np.where(train_padded != 0, 1, 0)
train_attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [ ]:
train_tokenized.values

array([list([101, 1037, 18385, 1010, 6057, 1998, 2633, 18276, 2128, 16603, 1997, 5053, 1998, 1996, 6841, 1998, 5687, 5469, 3152, 102]),
       list([101, 4593, 2128, 27241, 23931, 2013, 1996, 6276, 2282, 2723, 1997, 2151, 2445, 12217, 7815, 102]),
       list([101, 2027, 3653, 23545, 2037, 4378, 24185, 1050, 1005, 1056, 4133, 2145, 2005, 1037, 11507, 10800, 1010, 2174, 14036, 2135, 3591, 1010, 2061, 2027, 19817, 4140, 2041, 1996, 7511, 2671, 4349, 3787, 1997, 11829, 7168, 9219, 1998, 28971, 2308, 1999, 8301, 8737, 2100, 4253, 102]),
       ..., list([101, 2433, 20922, 2278, 1010, 2021, 4569, 102]),
       list([101, 2045, 1005, 1055, 2070, 2204, 3430, 1999, 2037, 2466, 2055, 1037, 7027, 7805, 5782, 2062, 2041, 1997, 2166, 1010, 2021, 1996, 3185, 2205, 2411, 23371, 2049, 7787, 2007, 5220, 8146, 1998, 23563, 5019, 102]),
       list([101, 11317, 7545, 1996, 5372, 10652, 2000, 2010, 2535, 2004, 1048, 15185, 4463, 15803, 25269, 2497, 102])],
      dtype=object)

In [ ]:
model.to(device)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
train_input_ids = torch.tensor(train_padded).long().to(device)
train_attention_mask = torch.tensor(train_attention_mask).long().to(device)

print(train_input_ids)
print('====='*10)
print(train_attention_mask)

tensor([[  101,  1037, 18385,  ...,     0,     0,     0],
        [  101,  4593,  2128,  ...,     0,     0,     0],
        [  101,  2027,  3653,  ...,     0,     0,     0],
        ...,
        [  101,  2433, 20922,  ...,     0,     0,     0],
        [  101,  2045,  1005,  ...,     0,     0,     0],
        [  101, 11317,  7545,  ...,     0,     0,     0]], device='cuda:0')
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')


In [ ]:
with torch.no_grad():
    train_last_hidden_states = model(train_input_ids, attention_mask=train_attention_mask)

In [ ]:
train_features = train_last_hidden_states[0][:,0,:].cpu().numpy()
train_labels = train_set[1]
train_features, test_features, train_labels, test_labels = train_test_split(train_features, train_labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)

0.8306666666666667

In [ ]:
max_len = 0
for i in train_set[0]:
   ids = tokenizer.encode(i, add_special_tokens=True)
   max_len = max(max_len, len(ids))

print(max_len)
input_ids = []
attention_masks = []
for x in train_set[0]:
    out = tokenizer.encode_plus(x, add_special_tokens=True,max_length=max_len,pad_to_max_length=True,
        return_attention_mask=True,  # Construct attn. masks.
        return_tensors='pt', truncation=True)
    input_ids.append(out['input_ids'])
    attention_masks.append(out['attention_mask'])

input_ids = torch.cat(input_ids, dim=0).to(device)
attention_mask = torch.cat(attention_masks, dim=0).to(device)
print(input_ids.shape, attention_mask.shape)

66
torch.Size([3000, 66]) torch.Size([3000, 66])


In [ ]:
with torch.no_grad():
  result = model(input_ids, attention_mask=attention_mask)
result[0].size()

torch.Size([3000, 66, 768])

In [ ]:
train_features = result[0][:,0,:].detach().cpu().numpy()
train_labels = train_set[1]

torch.cuda.empty_cache()

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(train_features, train_labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)

0.836

In [ ]:
!nvidia-smi

Tue Aug 17 09:37:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    72W / 149W |    918MiB / 11441MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
with torch.no_grad():
    train_last_hidden_states = model(train_input_ids.to(device), attention_mask=train_attention_mask.to(device))

In [ ]:
train_last_hidden_states[0].size()

torch.Size([3000, 66, 768])